In [2]:
from domino_data.vectordb import DominoPineconeConfiguration
from langchain.chains import RetrievalQA
from langchain_community.embeddings import MlflowEmbeddings
from langchain.vectorstores import Pinecone
from langchain import PromptTemplate
from langchain_community.chat_models import ChatMlflow

import csv
import os
import pinecone
import sys

from mlflow.deployments import get_deploy_client
import os

In [3]:
PINECONE_ENV="gcp-starter"
client = get_deploy_client(os.environ['DOMINO_MLFLOW_DEPLOYMENTS'])

In [5]:
# initialize embedding
embeddings = MlflowEmbeddings(
    target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
    endpoint="embedding-ada-002ja2"
)

In [6]:
# Domino Pinecone Data Source name 
datasource_name = "mrag-fin-docs-ja"

# Load Domino Pinecone Data Source Configuration 
conf = DominoPineconeConfiguration(datasource=datasource_name)

# api_key variable is a mandatory non-empty field used for 
# the native pinecone python client initialization 
# using Pinecone Data Source name 
api_key = os.environ.get("DOMINO_VECTOR_DB_METADATA", datasource_name)

# initialize pinecone
pinecone.init(
    api_key=api_key, 
    environment=PINECONE_ENV,
    openapi_config=conf # Domino Pinecone Data Source Configuration 
)

# Load Pinecone Index
index_name = "mrag-fin-docs"
index = pinecone.Index(index_name)
text_field = "text"

# switch back to normal index for langchain
vectorstore = Pinecone(
    index, embeddings.embed_query, text_field # Using embedded data from Domino AI Gateway Endpoint
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [7]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00361,
 'namespaces': {'': {'vector_count': 361}},
 'total_vector_count': 361}

In [17]:

prompt_template = """You are an AI assistant with expertise in financial analysis. You are given the following extracted parts and a question. 
Question: {question}
=========
{context}
=========
Answer in Markdown:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["question", "context"])
#
chain_type_kwargs = {"prompt": PROMPT}

In [18]:
rag_llm = ChatMlflow(
        target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
        endpoint="chat-gpt35turbo-ja", 
        temperature=0.0,
    )

In [19]:

qa_chain = RetrievalQA.from_chain_type(llm=rag_llm, # AI Gateway Endpoint
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=vectorstore.as_retriever(search_kwargs={"k": 20}), # Domino Pinecone Data Source
                                       return_source_documents=True
                                      )

In [24]:
user_question = input("Please ask your financial question:")
result = qa_chain(user_question)

Please ask your financial question: Report and analyze the trend in sales for the Americas and US from FY22 and FY23.


In [25]:
result['result']

'### Sales Trend Analysis for the Americas and US from FY22 to FY23\n\n#### Americas:\n- Net sales in the Americas decreased by 4% from $169.658 billion in FY22 to $162.560 billion in FY23.\n- The decrease was primarily due to lower net sales of iPhone and Mac, partially offset by higher net sales of Services.\n\n#### US:\n- Net sales in the US decreased by 6% from $147.859 billion in FY22 to $138.573 billion in FY23.\n- The decrease was driven by lower net sales of iPhone and Mac.\n\n### Analysis:\n- Both the Americas and US experienced a decline in net sales from FY22 to FY23.\n- The decrease in net sales in both regions was primarily attributed to lower sales of iPhone and Mac.\n- The increase in net sales of Services partially offset the decline in net sales of other products.\n- The weakness in foreign currencies relative to the US dollar also impacted net sales in these regions.\n\nOverall, the trend in sales for the Americas and US from FY22 to FY23 was negative, with a decrease

In [14]:
print(result)

{'query': 'How much revenue did the Americas generate in FY23?', 'result': 'To determine the revenue generated by the Americas in FY23, we can refer to the provided information from the segment operating performance section of the document. \n\nIn FY23, the net sales generated by the Americas segment were $162,560 million. This represented a 4% decrease from the previous year (FY22) when the net sales were $169,658 million. \n\nTherefore, the revenue generated by the Americas in FY23 was $162,560 million.', 'source_documents': [Document(page_content='Segment Operating Performance\nThe following table shows net sales by reportable segment for 2023, 2022 and 2021 (dollars in millions):\n2023 Change 2022 Change 2021\nNet sales by reportable segment:\nAmericas $ 162,560 (4)%$ 169,658 11 %$ 153,306 \nEurope 94,294 (1)% 95,118 7 % 89,307 \nGreater China 72,559 (2)% 74,200 9 % 68,366 \nJapan 24,257 (7)% 25,977 (9)% 28,482 \nRest of Asia Pacific 29,615 1 % 29,375 11 % 26,356 \nTotal net sales 